In [143]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopy.geocoders import Nominatim

In [2]:
df = pd.read_csv('global air pollution dataset.csv')
df_us = df[df['Country'] == 'United States of America']
df_us = df_us.drop(['Country'], axis=1).reset_index()
df_us = df_us.drop('index', axis=1)
df_us

,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category
0,Punta Gorda,54,Moderate,1,Good,14,Good,11,Good,54,Moderate
1,Phoenix,72,Moderate,1,Good,4,Good,23,Good,72,Moderate
2,Pontiac,77,Moderate,2,Good,22,Good,15,Good,77,Moderate
3,Port Neches,34,Good,1,Good,19,Good,7,Good,34,Good
4,Portales,77,Moderate,1,Good,34,Good,0,Good,77,Moderate
...,...,...,...,...,...,...,...,...,...,...,...
2867,Reston,60,Moderate,1,Good,40,Good,5,Good,60,Moderate
2868,Sand Springs,56,Moderate,1,Good,49,Good,2,Good,56,Moderate
2869,El Reno,39,Good,1,Good,39,Good,1,Good,35,Good
2870,Highland Springs,54,Moderate,1,Good,34,Good,5,Good,54,Moderate


In [69]:
geolocator = Nominatim(user_agent="air-pollution")

In [70]:
# New column geocode for geopy Location objects
df_us['geocode'] = df_us.apply(lambda row: geolocator.geocode(row.City), axis=1)

In [87]:
# drop rows with missing values
df_us = df_us.dropna(axis=0)

In [130]:
geometry = []
for index, row in df_us.iterrows():
    location = row.geocode
    geometry.append(Point(location.longitude, location.latitude))
gs = gpd.GeoSeries(geometry)
gdf_us = gpd.GeoDataFrame(df_us, geometry=gs, crs="EPSG:4326")
for index, row in gdf_us[gdf_us.isnull().any(axis=1)].iterrows():
    print(row.geocode)

Highland Springs, Lake County, California, United States
Westerville, Franklin County, Ohio, 43081, United States


In [132]:
gdf_us = gdf_us.dropna(axis=0)
gdf_us[gdf_us.isnull().any(axis=1)]

,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,geocode,geometry


In [133]:
gdf_us

,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,geocode,geometry
0,Punta Gorda,54,Moderate,1,Good,14,Good,11,Good,54,Moderate,"(Punta Gorda, Charlotte County, Florida, Unite...",POINT (-82.04537 26.92978)
1,Phoenix,72,Moderate,1,Good,4,Good,23,Good,72,Moderate,"(Phoenix, Maricopa County, Arizona, United Sta...",POINT (-112.07414 33.44844)
2,Pontiac,77,Moderate,2,Good,22,Good,15,Good,77,Moderate,"(Pontiac, Oakland County, Michigan, United Sta...",POINT (-83.29105 42.63892)
3,Port Neches,34,Good,1,Good,19,Good,7,Good,34,Good,"(Port Neches, Jefferson County, Texas, 77651, ...",POINT (-93.95851 29.99132)
4,Portales,77,Moderate,1,Good,34,Good,0,Good,77,Moderate,"(Portales, Roosevelt County, New Mexico, 88130...",POINT (-103.33440 34.18619)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2865,Union,51,Moderate,1,Good,47,Good,2,Good,51,Moderate,"(Union County, Illinois, United States, (37.46...",POINT (-77.34188 38.95570)
2866,Guaynabo,84,Moderate,0,Good,23,Good,0,Good,84,Moderate,"(Guaynabo, Puerto Rico, United States, (18.358...",POINT (-96.11000 36.13943)
2867,Reston,60,Moderate,1,Good,40,Good,5,Good,60,Moderate,"(Reston, Fairfax County, Virginia, United Stat...",POINT (-97.95506 35.53220)
2868,Sand Springs,56,Moderate,1,Good,49,Good,2,Good,56,Moderate,"(Sand Springs, Tulsa County, Oklahoma, United ...",POINT (-122.90694 38.93712)


In [141]:
gdf_us.to_file("us-cities-air-pollution.shp")

C:\Users\jrh25\AppData\Local\Temp\ipykernel_38384\555201581.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_us.to_file("us-cities-air-pollution.shp")


In [144]:
gdf_us

,City,AQI Value,AQI Category,CO AQI Value,CO AQI Category,Ozone AQI Value,Ozone AQI Category,NO2 AQI Value,NO2 AQI Category,PM2.5 AQI Value,PM2.5 AQI Category,geometry
0,Punta Gorda,54,Moderate,1,Good,14,Good,11,Good,54,Moderate,POINT (-82.04537 26.92978)
1,Phoenix,72,Moderate,1,Good,4,Good,23,Good,72,Moderate,POINT (-112.07414 33.44844)
2,Pontiac,77,Moderate,2,Good,22,Good,15,Good,77,Moderate,POINT (-83.29105 42.63892)
3,Port Neches,34,Good,1,Good,19,Good,7,Good,34,Good,POINT (-93.95851 29.99132)
4,Portales,77,Moderate,1,Good,34,Good,0,Good,77,Moderate,POINT (-103.33440 34.18619)
...,...,...,...,...,...,...,...,...,...,...,...,...
2865,Union,51,Moderate,1,Good,47,Good,2,Good,51,Moderate,POINT (-77.34188 38.95570)
2866,Guaynabo,84,Moderate,0,Good,23,Good,0,Good,84,Moderate,POINT (-96.11000 36.13943)
2867,Reston,60,Moderate,1,Good,40,Good,5,Good,60,Moderate,POINT (-97.95506 35.53220)
2868,Sand Springs,56,Moderate,1,Good,49,Good,2,Good,56,Moderate,POINT (-122.90694 38.93712)
